In [2]:
# JJ Rolling Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [7]:
bnf_1hr = pd.read_csv("../data/gold_4hr_tv (2).csv")
# bnf_1hr = pd.read_csv("../data/midcp_select_1hr_tv (4).csv")
# bnf_1hr = pd.read_csv("../data/fnf_1hr_tv.csv")
bnf_1hr["datetime"] = pd.to_datetime(bnf_1hr["time"])
bnf_1hr = bnf_1hr.drop(columns=["time"])
bnf_1hr = bnf_1hr[(bnf_1hr["datetime"].dt.year >= 2015)]
bnf_1hr.tail()

,open,high,low,close,datetime
9361,72650,72660,72130,72343,2024-07-08 21:00:00+05:30
9362,72577,72598,72310,72316,2024-07-09 09:00:00+05:30
9363,72327,72506,72311,72385,2024-07-09 13:00:00+05:30
9364,72377,72634,72111,72200,2024-07-09 17:00:00+05:30
9365,72217,72450,72191,72432,2024-07-09 21:00:00+05:30


In [8]:
# spot_data = pd.read_csv('../data/midcp_min.csv')
# spot_data['datetime'] = pd.to_datetime(spot_data['datetime'])
# spot_data = pl.DataFrame(spot_data)
# spot_data = spot_data.with_columns([pl.col('datetime').alias('index')])

# def resample(
#     data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
# ) -> pl.DataFrame:
#     return (
#         data.set_sorted("datetime")
#         .group_by_dynamic(
#             index_column="datetime",
#             every=timeframe,
#             period=timeframe,
#             label='left',
#             offset=offset,
#         )
#         .agg(
#             [
#                 pl.col("open").first().alias("open"),
#                 pl.col("high").max().alias("high"),
#                 pl.col("low").min().alias("low"),
#                 pl.col("close").last().alias("close"),
#                 pl.col("volume").sum().alias("volume"),
#             ]
#         )
#     )

# bnf_resampled = resample(spot_data, '15m', pd.Timedelta(minutes=0))

# bnf_df = bnf_resampled.to_pandas()
# bnf_1hr = bnf_df

In [10]:
PORTFOLIO = 10_00_000
INDEX_LEVERAGE = 5
SLIPPAGE_FACTOR = 0.0003

In [11]:
def generate_pivots2(df):

    df["Bullish Pivot"] = 0
    df["Bearish Pivot"] = 0
    # df["MA"] = df["close"].rolling(window=18).mean()

    # Generate signals using boolean masking
    bull_pivot_mask = (df["low"] >= df["low"].shift(1)) & (
        df["low"].shift(2) >= df["low"].shift(1)
    )
    bear_pivot_mask = (df["high"] <= df["high"].shift(1)) & (
        df["high"].shift(2) <= df["high"].shift(1)
    )

    df.loc[bull_pivot_mask, "Bullish Pivot"] = 1
    df.loc[bear_pivot_mask, "Bearish Pivot"] = 1

    return df


bnf_1hr = generate_pivots2(bnf_1hr)
bnf_1hr.head()

,open,high,low,close,datetime,Bullish Pivot,Bearish Pivot
0,26600,26769,26600,26700,2015-03-02 09:00:00+05:30,0,0
1,26709,26730,26653,26680,2015-03-02 13:00:00+05:30,0,0
2,26684,26742,26576,26619,2015-03-02 17:00:00+05:30,0,0
3,26623,26624,26500,26506,2015-03-02 21:00:00+05:30,0,1
4,26500,26525,26444,26463,2015-03-03 09:00:00+05:30,0,0


In [18]:
def calculate_rsi(df, column="close", period=22):

    delta = df[column].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi


bnf_1hr["RSI"] = calculate_rsi(bnf_1hr)

In [19]:
bnf_1hr.tail(50)

,open,high,low,close,datetime,Bullish Pivot,Bearish Pivot,RSI
9317,71784.0000,71880.0000,71540.0000,71594.0000,2024-06-21 21:00:00+05:30,0.0000,1.0000,54.9557
9318,71719.0000,71763.0000,71576.0000,71725.0000,2024-06-24 09:00:00+05:30,1.0000,0.0000,53.8393
9319,71723.0000,71858.0000,71531.0000,71661.0000,2024-06-24 13:00:00+05:30,0.0000,0.0000,47.9167
9320,71668.0000,71860.0000,71620.0000,71745.0000,2024-06-24 17:00:00+05:30,1.0000,0.0000,49.1069
9321,71770.0000,71825.0000,71660.0000,71772.0000,2024-06-24 21:00:00+05:30,0.0000,1.0000,47.7634
9322,71712.0000,71730.0000,71590.0000,71630.0000,2024-06-25 09:00:00+05:30,0.0000,0.0000,50.5824
9323,71648.0000,71917.0000,71565.0000,71750.0000,2024-06-25 13:00:00+05:30,0.0000,0.0000,53.2204
9324,71747.0000,71771.0000,71480.0000,71519.0000,2024-06-25 17:00:00+05:30,0.0000,1.0000,50.6843
9325,71519.0000,71597.0000,71366.0000,71460.0000,2024-06-25 21:00:00+05:30,0.0000,0.0000,52.0452
9326,71521.0000,71522.0000,71254.0000,71410.0000,2024-06-26 09:00:00+05:30,0.0000,0.0000,47.6508


In [20]:
# temp_df = bnf_1hr


def execute_trades(df):
    tradebook = []

    in_trade = False
    long_trade_active = False
    short_trade_active = False

    remark = ""

    signal_entry_price = 1_00_000
    signal_initial_sl = 0

    points = 0

    long_sl_pct = 1
    long_target_pct = 5

    short_sl_pct = 1
    short_target_pct = 5

    rsi_upper_range = 80
    rsi_lower_range = 45

    long_points = 0
    short_points = 0

    for i in range(0, len(df)):

        points = 0

        previous_candle_open = df.iloc[i - 1]["open"]
        previous_candle_high = df.iloc[i - 1]["high"]
        previous_candle_low = df.iloc[i - 1]["low"]
        previous_candle_close = df.iloc[i - 1]["close"]
        previous_candle_time = df.iloc[i - 1]["datetime"]
        previous_candle = df.iloc[i - 1]

        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_candle_time = df.iloc[i]["datetime"]
        current_candle = df.iloc[i]

        # print(
        #     current_candle_time,
        #     current_candle_open,
        #     current_candle_high,
        #     current_candle_low,
        #     current_candle_close,
        # )

        if not in_trade:

            if (df.iloc[i - 1]["Bullish Pivot"] == 1) & (
                df.iloc[i - 1]["Bearish Pivot"] == 1
            ):
                if df.iloc[i]["RSI"] >= rsi_upper_range:
                    df.at[i - 1, "Bullish Pivot"] = 0
                elif df.iloc[i]["RSI"] <= rsi_lower_range:
                    df.at[i - 1, "Bearish Pivot"] = 0
                else:
                    df.at[i - 1, "Bullish Pivot"] = 0
                    df.at[i - 1, "Bearish Pivot"] = 0

            if df.iloc[i - 1]["Bullish Pivot"] == 1:
                # Long Side
                signal_time = previous_candle_time

                if current_candle_high >= previous_candle_high and (
                    df.iloc[i - 1]["RSI"] <= rsi_lower_range
                ):
                    if current_candle_open > previous_candle_high:
                        # Check Entry Skip
                        # print("Long Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        # print("Long Entry Triggered", current_candle_time)
                        in_trade = True
                        long_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_high
                        initial_sl = min(
                            df.iloc[i - 2]["low"], entry_price * (1 - long_sl_pct / 100)
                        )
                        long_target = entry_price * (1 + long_target_pct / 100)
                        qty = PORTFOLIO * INDEX_LEVERAGE / entry_price

            if df.iloc[i - 1]["Bearish Pivot"] == 1:
                # Short Side
                signal_time = previous_candle_time

                if current_candle_low <= previous_candle_low and (
                    df.iloc[i - 1]["RSI"] >= rsi_upper_range
                ):
                    if current_candle_open < previous_candle_low:
                        # Check Entry Skip
                        # print("Short Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        # print("Short Entry Triggered", current_candle_time)
                        in_trade = True
                        short_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_low
                        initial_sl = max(
                            df.iloc[i - 2]["high"],
                            entry_price * (1 + short_sl_pct / 100),
                        )
                        short_target = entry_price * (1 - short_target_pct / 100)
                        qty = PORTFOLIO * INDEX_LEVERAGE / entry_price * 0.1

        if in_trade:
            if long_trade_active:
                if current_candle_open < initial_sl:
                    # Gap Outside Initial SL
                    in_trade = False
                    long_trade_active = False
                    exit_price = current_candle_open
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Gap Outside Initial SL"
                    # print(remark, current_candle_time, exit_price)

                elif (
                    (df.iloc[i - 1]["Bearish Pivot"] == 1)
                    and (current_candle_low <= previous_candle_low)
                    and (not (current_candle_open < previous_candle_low))
                    and (df.iloc[i - 2]["RSI"] >= rsi_upper_range)
                ):
                    # if (current_candle_low < previous_candle_low) and (not(current_candle_open < previous_candle_low)): # Ensuring that there was no Gap for entry
                    # Reversal Trade
                    exit_price = previous_candle_low
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Reversed"
                    # print(remark, current_candle_time, exit_price)
                    trade = {
                        "Trade Type": "LONG",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": long_points,
                        "Points w CS": long_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * long_points,
                        "PnL w CS": qty
                        * (
                            long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * long_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    long_points = 0

                    long_trade_active = False
                    short_trade_active = True
                    in_trade = True

                    entry_price = previous_candle_low
                    initial_sl = max(
                        df.iloc[i - 2]["high"], entry_price * (1 + short_sl_pct / 100)
                    )
                    short_target = entry_price * (1 - short_target_pct / 100)
                    signal_time = previous_candle_time
                    entry_time = current_candle_time
                    # print(signal_time, entry_time, entry_price, initial_sl)
                    qty = PORTFOLIO * INDEX_LEVERAGE / entry_price * 0.1
                    # continue

                    if current_candle_high >= initial_sl:
                        if not (
                            current_candle_open > initial_sl
                        ):  # Ensuring that there was no Gap for entry
                            # Initial SL Hit
                            # print(current_candle)
                            in_trade = False
                            long_trade_active = False
                            exit_price = initial_sl
                            exit_time = current_candle_time
                            short_points = entry_price - exit_price
                            remark = "Initial SL Hit"
                            # print(remark, current_candle_time, exit_price)

                            trade = {
                                "Trade Type": "SHORT",
                                "Signal Generated At": signal_time,
                                "Entry Time": entry_time,
                                "Entry Price": entry_price,
                                "Initial SL": initial_sl,
                                "Exit Time": exit_time,
                                "Exit Price": exit_price,
                                "Remarks": remark,
                                "Points Captured": short_points,
                                "Points w CS": short_points
                                - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                                "Qty": qty,
                                "Leverage": INDEX_LEVERAGE,
                                "PnL": qty * short_points,
                                "PnL w CS": qty
                                * (
                                    short_points
                                    - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                                ),
                                "ROI%": qty * short_points * 100 / PORTFOLIO,
                                "ROI% w CS": qty
                                * (
                                    short_points
                                    - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                                )
                                * 100
                                / PORTFOLIO,
                                "Trade Year": entry_time.year,
                            }
                            tradebook.append(trade)

                            short_points = 0
                            short_trade_active = False

                elif current_candle_low <= initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    long_trade_active = False
                    exit_price = initial_sl
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Initial SL Hit"
                    # print(remark, current_candle_time, exit_price)

                elif current_candle_high >= long_target:
                    in_trade = False
                    long_trade_active = False
                    exit_price = long_target
                    exit_time = current_candle_time
                    long_points = exit_price - entry_price
                    remark = "Target Hit"
                    # print(remark, current_candle_time, exit_price)

                if long_points:
                    trade = {
                        "Trade Type": "LONG",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": long_points,
                        "Points w CS": long_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * long_points,
                        "PnL w CS": qty
                        * (
                            long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * long_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    long_points = 0
                    long_trade_active = False

            if short_trade_active:
                if current_candle_open >= initial_sl:
                    # Gap Outside Initial SL
                    in_trade = False
                    short_trade_active = False
                    exit_price = current_candle_open
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Gap Outside Initial SL"
                    # print(remark, current_candle_time, exit_price)

                elif (
                    (df.iloc[i - 1]["Bullish Pivot"] == 1)
                    and (current_candle_high >= previous_candle_high)
                    and (not (current_candle_open > previous_candle_high))
                    and (df.iloc[i - 2]["RSI"] <= rsi_lower_range)
                ):
                    # if (current_candle_high > previous_candle_high) and (not(current_candle_open > previous_candle_high)): # Ensuring that there was no Gap for entry
                    # Reversal Tradce
                    exit_price = previous_candle_high
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Reversed"
                    # print(remark, current_candle_time, exit_price)
                    trade = {
                        "Trade Type": "SHORT",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": short_points,
                        "Points w CS": short_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * short_points,
                        "PnL w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * short_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        )
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    short_points = 0

                    long_trade_active = True
                    short_trade_active = False
                    in_trade = True

                    entry_price = previous_candle_high
                    initial_sl = min(
                        df.iloc[i - 2]["low"], entry_price * (1 - long_sl_pct / 100)
                    )
                    long_target = entry_price * (1 + long_target_pct / 100)
                    signal_time = previous_candle_time
                    entry_time = current_candle_time
                    qty = PORTFOLIO * INDEX_LEVERAGE / entry_price
                    # print(signal_time, entry_time, entry_price, initial_sl)
                    # continue

                    if current_candle_low <= initial_sl:
                        if not (
                            current_candle_open < initial_sl
                        ):  # Ensuring that there was no Gap for entry
                            # Initial SL Hit
                            # print(current_candle)
                            in_trade = False
                            short_trade_active = False
                            exit_price = initial_sl
                            exit_time = current_candle_time
                            long_points = exit_price - entry_price
                            remark = "Initial SL Hit"
                            # print(remark, current_candle_time, exit_price)

                            trade = {
                                "Trade Type": "LONG",
                                "Signal Generated At": signal_time,
                                "Entry Time": entry_time,
                                "Entry Price": entry_price,
                                "Initial SL": initial_sl,
                                "Exit Time": exit_time,
                                "Exit Price": exit_price,
                                "Remarks": remark,
                                "Points Captured": long_points,
                                "Points w CS": long_points
                                - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                                "Qty": qty,
                                "Leverage": INDEX_LEVERAGE,
                                "PnL": qty * long_points,
                                "PnL w CS": qty
                                * (
                                    long_points
                                    - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                                ),
                                "ROI%": qty * long_points * 100 / PORTFOLIO,
                                "ROI% w CS": qty
                                * (
                                    long_points
                                    - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                                )
                                * 100
                                / PORTFOLIO,
                                "Trade Year": entry_time.year,
                            }
                            tradebook.append(trade)

                            long_points = 0
                            long_trade_active = False

                elif current_candle_high >= initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    short_trade_active = False
                    exit_price = initial_sl
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Initial SL Hit"
                    # print(remark, current_candle_time, exit_price)

                elif current_candle_low <= short_target:
                    in_trade = False
                    short_trade_active = False
                    exit_price = short_target
                    exit_time = current_candle_time
                    short_points = entry_price - exit_price
                    remark = "Target Hit"
                    # print(remark, current_candle_time, exit_price)

                if short_points:
                    trade = {
                        "Trade Type": "SHORT",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": short_points,
                        "Points w CS": short_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * short_points,
                        "PnL w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * short_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (
                            short_points
                            - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        )
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    short_points = 0
                    short_trade_active = False

    trade_book_df = pd.DataFrame(tradebook)
    return trade_book_df


tb = execute_trades(bnf_1hr)

In [26]:
tb

,Trade Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Remarks,Points Captured,Points w CS,Qty,Leverage,PnL,PnL w CS,ROI%,ROI% w CS,Trade Year
0,LONG,2015-03-03 13:00:00+05:30,2015-03-03 17:00:00+05:30,26533.0000,26267.6700,2015-03-06 17:00:00+05:30,26267.6700,Initial SL Hit,-265.3300,-281.1702,188.4446,5,-50000.0000,-52985.0000,-5.0000,-5.2985,2015
1,LONG,2015-03-09 09:00:00+05:30,2015-03-09 13:00:00+05:30,26096.0000,25835.0400,2015-03-10 09:00:00+05:30,25835.0400,Initial SL Hit,-260.9600,-276.5393,191.6002,5,-50000.0000,-52985.0000,-5.0000,-5.2985,2015
2,LONG,2015-03-11 21:00:00+05:30,2015-03-12 09:00:00+05:30,25750.0000,25492.5000,2015-04-06 13:00:00+05:30,27037.5000,Target Hit,1287.5000,1271.6637,194.1748,5,250000.0000,246925.0000,25.0000,24.6925,2015
3,LONG,2015-04-15 17:00:00+05:30,2015-04-15 21:00:00+05:30,26621.0000,26354.7900,2015-07-02 09:00:00+05:30,26354.7900,Initial SL Hit,-266.2100,-282.1027,187.8216,5,-50000.0000,-52985.0000,-5.0000,-5.2985,2015
4,LONG,2015-07-08 13:00:00+05:30,2015-07-08 17:00:00+05:30,26080.0000,25819.2000,2015-07-15 17:00:00+05:30,25819.2000,Initial SL Hit,-260.8000,-276.3698,191.7178,5,-50000.0000,-52985.0000,-5.0000,-5.2985,2015
5,SHORT,2015-08-12 13:00:00+05:30,2015-08-12 17:00:00+05:30,25743.0000,26000.4300,2015-08-12 21:00:00+05:30,26000.4300,Initial SL Hit,-257.4300,-272.9530,19.4228,5,-5000.0000,-5301.5000,-0.5000,-0.5302,2015
6,SHORT,2015-08-13 09:00:00+05:30,2015-08-13 13:00:00+05:30,25866.0000,26124.6600,2015-08-19 17:00:00+05:30,26124.6600,Initial SL Hit,-258.6600,-274.2572,19.3304,5,-5000.0000,-5301.5000,-0.5000,-0.5301,2015
7,LONG,2015-08-28 21:00:00+05:30,2015-08-31 09:00:00+05:30,26685.0000,26381.0000,2015-09-03 17:00:00+05:30,26381.0000,Initial SL Hit,-304.0000,-319.9198,187.3712,5,-56960.8394,-59943.7512,-5.6961,-5.9944,2015
8,LONG,2015-10-01 13:00:00+05:30,2015-10-01 17:00:00+05:30,26054.0000,25793.4600,2015-10-15 17:00:00+05:30,27356.7000,Target Hit,1302.7000,1286.6768,191.9091,5,250000.0000,246925.0000,25.0000,24.6925,2015
9,LONG,2015-10-22 21:00:00+05:30,2015-10-23 09:00:00+05:30,26923.0000,26653.7700,2015-10-29 17:00:00+05:30,26653.7700,Initial SL Hit,-269.2300,-285.3030,185.7148,5,-50000.0000,-52985.0000,-5.0000,-5.2985,2015


In [21]:
tb["ROI% w CS"].sum()

569.1915967455639

In [22]:
stats_df8 = pd.DataFrame(
    index=range(2015, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb
# Iterate over each year
for year in range(2015, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w CS"] > 0][
    "ROI% w CS"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w CS"] < 0][
    "ROI% w CS"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w CS"].cumsum()
    - combined_df_sorted["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2015,24.3086,13,23.0769,22.8176,-4.4144,-17.6517,1.3771
2016,72.8636,22,31.8182,21.1731,-5.0232,-34.2302,2.1286
2017,66.5184,15,40.0000,19.0342,-5.2985,-26.4925,2.5108
2018,71.4912,10,40.0000,24.6925,-4.5465,-10.5970,6.7464
2019,77.0694,16,50.0000,13.7401,-4.1064,-15.8955,4.8485
2020,113.3793,15,40.0000,24.6925,-3.8640,-17.1897,6.5958
2021,7.2348,21,23.8095,17.7644,-5.0992,-38.6690,0.1871
2022,59.4098,17,29.4118,23.6447,-4.9011,-21.1940,2.8031
2023,40.2983,16,31.2500,20.0547,-5.4523,-28.1844,1.4298
2024,36.6181,9,33.3333,16.5182,-2.1561,-6.5777,5.5670


# Gold Qty (L 1: S 0.1) Rolling Pivots + RSI : RSI(period = 21) UR: 70 , LR: 40 , SL 1% , Target 5%

In [23]:
# bnf_1hr.tail(99)

In [24]:
tb_shorts = tb[tb["Trade Type"] == "SHORT"]
# tb_shorts['ROI% w CS'].sum()
tb_longs = tb[tb["Trade Type"] == "LONG"]

tb_longs["ROI% w CS"].sum(), tb_shorts["ROI% w CS"].sum()

(568.765071385297, 0.42652536026706156)

In [25]:
stats_df8 = pd.DataFrame(
    index=range(2015, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
# combined_df_sorted = tb_shorts
combined_df_sorted = tb_longs

# Iterate over each year
for year in range(2015, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w CS"] > 0][
    "ROI% w CS"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w CS"] < 0][
    "ROI% w CS"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w CS"].cumsum()
    - combined_df_sorted["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2015,25.3689,11,27.2727,22.8176,-5.3855,-16.5914,1.5290
2016,73.3368,20,30.0000,24.6925,-5.3442,-34.2302,2.1425
2017,66.1424,14,35.7143,22.7658,-5.2985,-26.4925,2.4966
2018,72.2775,9,44.4444,24.6925,-5.2985,-10.5970,6.8206
2019,73.8530,11,45.4545,21.1288,-5.2985,-15.8955,4.6462
2020,114.9698,12,50.0000,24.6925,-5.5309,-16.1294,7.1280
2021,5.2942,19,21.0526,21.5878,-5.4038,-32.7106,0.1618
2022,59.9400,16,31.2500,23.6447,-5.2985,-21.1940,2.8282
2023,38.7946,15,26.6667,24.6925,-5.4523,-28.1844,1.3765
2024,38.7880,4,50.0000,24.6925,-5.2985,-5.2985,7.3206


In [1239]:
tb_longs

,Trade Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Remarks,Points Captured,Points w CS,Qty,Leverage,PnL,PnL w CS,ROI%,ROI% w CS,Trade Year
0,LONG,2015-03-12 09:00:00+05:30,2015-03-12 13:00:00+05:30,3047,2986.0600,2015-03-12 17:00:00+05:30,2986.0600,Initial SL Hit,-60.9400,-63.9565,1640.9583,5,-100000.0000,-104950.0000,-10.0000,-10.4950,2015
1,LONG,2015-03-13 09:00:00+05:30,2015-03-13 13:00:00+05:30,2974,2914.5200,2015-03-13 17:00:00+05:30,2914.5200,Initial SL Hit,-59.4800,-62.4243,1681.2374,5,-100000.0000,-104950.0000,-10.0000,-10.4950,2015
2,LONG,2015-05-20 21:00:00+05:30,2015-05-21 09:00:00+05:30,3795,3719.1000,2015-05-26 21:00:00+05:30,3719.1000,Initial SL Hit,-75.9000,-79.6571,1317.5231,5,-100000.0000,-104950.0000,-10.0000,-10.4950,2015
3,LONG,2015-05-28 21:00:00+05:30,2015-05-29 09:00:00+05:30,3679,3605.4200,2015-07-03 09:00:00+05:30,3605.4200,Initial SL Hit,-73.5800,-77.2222,1359.0650,5,-100000.0000,-104950.0000,-10.0000,-10.4950,2015
4,LONG,2015-07-03 13:00:00+05:30,2015-07-03 17:00:00+05:30,3604,3531.9200,2015-07-03 21:00:00+05:30,3531.9200,Initial SL Hit,-72.0800,-75.6480,1387.3474,5,-100000.0000,-104950.0000,-10.0000,-10.4950,2015
5,LONG,2015-07-09 09:00:00+05:30,2015-07-09 13:00:00+05:30,3353,3245.0000,2015-07-14 09:00:00+05:30,3245.0000,Initial SL Hit,-108.0000,-111.2990,1491.2019,5,-161049.8061,-165969.2812,-16.1050,-16.5969,2015
6,LONG,2015-07-14 17:00:00+05:30,2015-07-14 21:00:00+05:30,3333,3235.0000,2015-07-17 09:00:00+05:30,3235.0000,Initial SL Hit,-98.0000,-101.2840,1500.1500,5,-147014.7015,-151941.1941,-14.7015,-15.1941,2015
7,LONG,2015-07-24 09:00:00+05:30,2015-07-24 13:00:00+05:30,3138,3075.2400,2015-07-27 13:00:00+05:30,3075.2400,Initial SL Hit,-62.7600,-65.8666,1593.3716,5,-100000.0000,-104950.0000,-10.0000,-10.4950,2015
8,LONG,2015-07-28 17:00:00+05:30,2015-07-28 21:00:00+05:30,3097,3008.0000,2015-08-03 09:00:00+05:30,3008.0000,Initial SL Hit,-89.0000,-92.0525,1614.4656,5,-143687.4395,-148615.5957,-14.3687,-14.8616,2015
9,LONG,2015-08-04 09:00:00+05:30,2015-08-04 13:00:00+05:30,2934,2875.3200,2015-08-05 21:00:00+05:30,2875.3200,Initial SL Hit,-58.6800,-61.5847,1704.1581,5,-100000.0000,-104950.0000,-10.0000,-10.4950,2015


# GOLD Long Only RSI(period=8) + Pivots 70, 40 1 : 4

In [653]:
# stats_df8 = pd.DataFrame(
#     index=range(2017, 2025),
#     columns=[
#         "Total ROI",
#         "Total Trades",
#         "Win Rate",
#         "Avg Profit% per Trade",
#         "Avg Loss% per Trade",
#         "Max Drawdown",
#         "ROI/DD Ratio",
#     ],
# )
# combined_df_sorted = tb_shorts
# # combined_df_sorted = tb_longs

# # Iterate over each year
# for year in range(2017, 2025):
#     # Filter trades for the current year
#     year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

#     # Calculate total ROI
#     total_roi = year_trades["ROI% w CS"].sum()

#     # Calculate total number of trades
#     total_trades = len(year_trades)

#     # Calculate win rate
#     win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

#     # Calculate average profit per trade
#     avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

#     # Calculate average loss per trade
#     avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

#     # Calculate maximum drawdown
#     max_drawdown = (
#         year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
#     ).min()

#     # Calculate ROI/DD ratio
#     roi_dd_ratio = total_roi / abs(max_drawdown)

#     # Store the statistics in the DataFrame
#     stats_df8.loc[year] = [
#         total_roi,
#         total_trades,
#         win_rate,
#         avg_profit,
#         avg_loss,
#         max_drawdown,
#         roi_dd_ratio,
#     ]

# # Calculate overall statistics
# overall_total_roi = stats_df8["Total ROI"].sum()
# overall_total_trades = stats_df8["Total Trades"].sum()
# overall_win_rate = (combined_df_sorted["ROI% w CS"] > 0).mean() * 100
# overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w CS"] > 0][
#     "ROI% w CS"
# ].mean()
# overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w CS"] < 0][
#     "ROI% w CS"
# ].mean()
# overall_max_drawdown = (
#     combined_df_sorted["ROI% w CS"].cumsum()
#     - combined_df_sorted["ROI% w CS"].cumsum().cummax()
# ).min()
# overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# # Store the overall statistics in the DataFrame
# stats_df8.loc["Overall"] = [
#     overall_total_roi,
#     overall_total_trades,
#     overall_win_rate,
#     overall_avg_profit,
#     overall_avg_loss,
#     overall_max_drawdown,
#     overall_roi_dd_ratio,
# ]
# stats_df8

In [280]:
# mtrend = pd.read_csv('../bnf_systems/BNF_48_1_18_4X.csv')
# mtrend['Entry Time'] = pd.to_datetime(mtrend['Entry Time'])
# mtrend['Final ROI'] = mtrend['ROI%']
# mtrend['Final ROI'] *= 0.65
# mtrend['TAG'] = 'MTrend'

# reversals = tb
# reversals['Entry Time'] = reversals['Entry Time'].dt.tz_localize(None)
# reversals['Final ROI'] = reversals['ROI% w CS']
# reversals['Final ROI'] *=0.35
# reversals['TAG'] = 'Reversal'

# comb = pd.concat([mtrend, reversals], ignore_index=True)
# comb['Entry Time'] = pd.to_datetime(comb['Entry Time'])
# comb = comb.sort_values(by="Entry Time")

In [281]:
# comb['Final ROI'] /= 2
comb["DD%"] = comb["Final ROI"].cumsum() - comb["Final ROI"].cumsum().cummax()
comb.drop(columns=["Unnamed: 0"], inplace=True)
# comb

NameError: name 'comb' is not defined

In [468]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = comb
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Final ROI"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Final ROI"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Final ROI"] > 0]["Final ROI"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Final ROI"] < 0]["Final ROI"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Final ROI"].cumsum() - year_trades["Final ROI"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Final ROI"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Final ROI"] > 0][
    "Final ROI"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Final ROI"] < 0][
    "Final ROI"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Final ROI"].cumsum()
    - combined_df_sorted["Final ROI"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,41.7696,192,35.9375,2.0989,-0.8379,-14.3688,2.9070
2018,24.3529,190,35.7895,1.9588,-0.8922,-22.6954,1.0730
2019,94.4055,162,39.5062,2.9283,-0.9491,-13.4991,6.9934
2020,185.5689,158,35.4430,6.0099,-1.4803,-23.1976,7.9995
2021,108.0920,170,36.4706,3.8147,-1.1891,-22.7831,4.7444
2022,61.5872,197,35.5330,2.8771,-1.1009,-21.4403,2.8725
2023,37.5102,162,40.7407,1.7036,-0.7805,-12.1111,3.0972
2024,11.4772,74,29.7297,2.6002,-0.8794,-14.4379,0.7949
Overall,564.7634,1305.0000,36.5517,2.9550,-1.0203,-23.1976,24.3457


In [469]:
comb.to_csv("Check MTrend_w_Reversals TB.csv")